# generate data

In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import math
import random

random.seed(0)
random_factor = 0.05
steps_per_cycle = 80
num_of_cycles = 50

df = pd.DataFrame(np.arange(steps_per_cycle * num_of_cycles + 1), columns=["t"])
df["sin_t"] = df.t.apply(lambda x: math.sin(x * (2 * math.pi / steps_per_cycle) + random.uniform(-1.0, +1.0) * random_factor))

print(df.head())
df["sin_t"].head(steps_per_cycle * 2).plot()

# define data extraction function

In [ ]:
def load_data(data, n_prev=100):
    docX, docY = [], []
    for i in range(len(data) - n_prev):
        docX.append(data[i:i + n_prev].values)
        docY.append(data[i + n_prev: i + n_prev + 1].values)
    alsX = np.array(docX)
    alsY = np.array(docY)
    
    return alsX, alsY

def train_test_split(df, test_size=0.1, n_prev=100):
    ntrn = int(round(len(df) * (1 - test_size)))
    X_train, y_train = load_data(df.iloc[0:ntrn], n_prev)
    X_test, y_test = load_data(df.iloc[ntrn:], n_prev)
    
    return (X_train, y_train), (X_test, y_test)

# extract data

In [ ]:
length_of_sequences = 100
(X_train, y_train), (X_test, y_test) = train_test_split(df["sin_t"], n_prev=length_of_sequences)

# generate NN model(4 layer)

In [ ]:
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers.core import Dense

nn_in = 100
nn_hid = 10
nn_out = 1
model = Sequential()
model.add(InputLayer(input_shape=(nn_in,)))
model.add(Dense(nn_hid, activation="tanh", bias=False))
model.add(Dense(nn_hid, activation="tanh", bias=False))
model.add(Dense(nn_out, activation="linear", bias=False))
model.compile(loss="mean_squared_error", optimizer="Adam")
model.summary()

# predict

In [ ]:
hist = model.fit(X_train, y_train, batch_size=600, nb_epoch=10, validation_split=0.05)

# check predicted data

In [ ]:
predicted = model.predict(X_test)
dataf = pd.DataFrame(predicted[:200])
dataf.columns = ["predict"]
dataf["input"] = y_test[:200]
dataf.plot(figsize=(15, 5)).grid()

# display model diagram

In [ ]:
from IPython.display import SVG
from keras.utils.visualize_util import model_to_dot, plot

SVG(model_to_dot(model, show_shapes=True).create(prog="dot", format="svg"))

# display loss graph

In [ ]:
datal = pd.DataFrame(hist.history)
plt = datal.plot()
plt.legend(["train", "validate"])
plt.grid()